In [182]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import io
from scipy import stats
from statsmodels import robust
%matplotlib inline

In [183]:
df = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto29/Cuarentenas-Totales.csv?raw=true', sep=',')

In [184]:
df.head()

,ID,Nombre,Estado,Alcance,Fecha de Inicio,Fecha de Término,Código CUT Comuna,Detalle,Superficie en m2,Perímetro en m,Region,n_REGION
0,1,Arica,Histórica,Área Urbana Completa,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,Corresponde al Radio Urbano de la Ciudad de Ar...,28050418.80,48722.30001,15,R15
1,2,Santiago Norte,Histórica,Sector Específico,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,Corresponde a la extensión de la cuarentena en...,19939218.57,21599.35134,13,R13
2,3,Puente Alto Poniente,Histórica,Sector Específico,2020-04-09 22:00:00,2020-05-08 22:00:00,13201,Corresponde al sector delimitado por la Av. Co...,42364268.21,30428.05152,13,R13
3,4,El Bosque,Histórica,Comuna completa,2020-04-16 22:00:00,2020-09-21 05:00:00,13105,Aplicada a la totalidad de la comuna. Se incor...,20622713.23,18788.87075,13,R13
4,5,San Bernardo Nororiente,Histórica,Sector Específico,2020-04-16 22:00:00,2020-05-08 22:00:00,13401,Corresponde al sector delimitado por los limit...,16326216.83,24929.14572,13,R13


In [185]:
df.drop(['Estado','Alcance','Detalle', 'Superficie en m2',
       'Perímetro en m','Region','n_REGION'], axis=1, inplace=True)

In [186]:
df1 = pd.read_csv('https://github.com/MinCiencia/Datos-COVID19/blob/master/output/producto1/Covid-19.csv?raw=true')

In [187]:
df1.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,2020-03-30,2020-04-01,2020-04-03,2020-04-06,2020-04-08,...,2021-09-10,2021-09-13,2021-09-17,2021-09-20,2021-09-24,2021-09-27,2021-10-01,2021-10-04,2021-10-08,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,6.0,6.0,12.0,41.0,63.0,...,29763.0,29836.0,29929.0,30002.0,30128.0,30239.0,30376.0,30463.0,30536.0,12335.2
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,0.0,0.0,0.0,0.0,0.0,...,67.0,67.0,70.0,70.0,70.0,70.0,70.0,70.0,70.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,0.0,0.0,0.0,0.0,0.0,...,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,88.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,0.0,0.0,0.0,0.0,0.0,...,205.0,207.0,208.0,207.0,211.0,215.0,213.0,213.0,213.0,8469.2
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,158.0,163.0,161.0,163.0,165.0,165.0,167.0,170.0,175.0,NaN


In [188]:
df_demiinteres= df1[['Region','Codigo region','Comuna','Codigo comuna','Poblacion','Tasa']]

In [189]:
df_demiinteres.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Tasa
0,Arica y Parinacota,15,Arica,15101.0,247552.0,12335.2
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,5677.2
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,10864.2
3,Arica y Parinacota,15,Putre,15201.0,2515.0,8469.2
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,NaN


In [190]:
df_demiinteres=df_demiinteres.dropna()

In [191]:
df_demiinteres['Codigo comuna'].astype(int)

0      15101
1      15102
2      15202
3      15201
5       1107
       ...  
356    12101
357    12103
358    12104
359    12303
360    12402
Name: Codigo comuna, Length: 346, dtype: int32

In [192]:
df=df.rename(columns={'Código CUT Comuna':'Codigo comuna'})

In [193]:
dfn=df.merge(df_demiinteres, on='Codigo comuna')

In [194]:
dfn.head()

,ID,Nombre,Fecha de Inicio,Fecha de Término,Codigo comuna,Region,Codigo region,Comuna,Poblacion,Tasa
0,1,Arica,2020-04-16 22:00:00,2020-05-15 22:00:00,15101,Arica y Parinacota,15,Arica,247552.0,12335.2
1,99,Arica Re-Ingreso urbano,2020-07-14 22:00:00,2020-11-09 05:00:00,15101,Arica y Parinacota,15,Arica,247552.0,12335.2
2,2,Santiago Norte,2020-04-13 05:00:00,2020-05-05 22:00:00,13101,Metropolitana,13,Santiago,503147.0,9009.3
3,18,Santiago,2020-03-26 21:00:00,2020-04-13 05:00:00,13101,Metropolitana,13,Santiago,503147.0,9009.3
4,40,Santiago Extensión a Total,2020-05-05 22:00:00,2020-08-17 05:00:00,13101,Metropolitana,13,Santiago,503147.0,9009.3


In [195]:
dfn.drop(['Nombre','ID','Codigo comuna','Region','Tasa'], axis=1, inplace=True)

In [196]:
dfn.head()

,Fecha de Inicio,Fecha de Término,Codigo region,Comuna,Poblacion
0,2020-04-16 22:00:00,2020-05-15 22:00:00,15,Arica,247552.0
1,2020-07-14 22:00:00,2020-11-09 05:00:00,15,Arica,247552.0
2,2020-04-13 05:00:00,2020-05-05 22:00:00,13,Santiago,503147.0
3,2020-03-26 21:00:00,2020-04-13 05:00:00,13,Santiago,503147.0
4,2020-05-05 22:00:00,2020-08-17 05:00:00,13,Santiago,503147.0


In [197]:
dfn['Fecha de Inicio'] = dfn['Fecha de Inicio'].apply(lambda x: x.split(' ')[0])
dfn['Fecha de Término'] = dfn['Fecha de Término'].apply(lambda x: x.split(' ')[0])

In [198]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fecha de Inicio   187 non-null    object 
 1   Fecha de Término  187 non-null    object 
 2   Codigo region     187 non-null    int64  
 3   Comuna            187 non-null    object 
 4   Poblacion         187 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 8.8+ KB


In [199]:
dfn['Fecha de Inicio'] = pd.to_datetime(df['Fecha de Inicio'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')
dfn['Fecha de Término'] = pd.to_datetime(df['Fecha de Término'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')

In [232]:
dfn.head()

,Fecha de Inicio,Fecha de Término,Region,Comuna,Poblacion
0,16-04-2020,15-05-2020,15,Arica,247552.0
1,13-04-2020,05-05-2020,15,Arica,247552.0
2,09-04-2020,08-05-2020,13,Santiago,503147.0
3,16-04-2020,21-09-2020,13,Santiago,503147.0
4,16-04-2020,08-05-2020,13,Santiago,503147.0


In [233]:
dfn=dfn.rename(columns={'Codigo region':'Region'})

SyntaxError: invalid syntax (<ipython-input-241-e44f4f9e658a>, line 1)

In [240]:
plt.figure(figsize=(15,8))
sns.lineplot(x=, y='Poblacion',data=dfn)

NameError: name 'm' is not defined

<Figure size 1080x576 with 0 Axes>